Example 30 - Comparison of Timoshenko and Euler-Bernoulli Beam Theory Results for Self-Aligning Bearings in Rotating Solid and Hollow Shafts
=====
This example is based on 5.11 Problems - 5.1  from {cite}`friswell2010dynamics`.

A uniform steel shaft, shown in Figure 5.48, is 1.6 m long and supported by plain, self-aligning bearings at its ends. It carries two identical disks, each with a diameter of 400 mm and a thickness of 80 mm. The disks are located 0.5 m and 1.0 m from the left bearing. For steel, E = 200 GPa, v = 0.27, and p = 7,800 kg/m3. 

Using both Timoshenko and Euler-Bernoulli beam models with 16 elements, determine the first five natural frequencies for the systems with the following shafts, spinning at 3,000 rev/min:
 (a) a solid shaft of 60 mm diameter
 (b) a hollow shaft with 80 mm outside diameter and 73 mm inside diameter 

Show that the percentage difference between the Timoshenko and Euler Bernoulli results for the rotor with the hollow shaft is about five times greater than for the rotor with the smaller diameter solid shaft. The shaft diameters are chosen to give similar shaft stiffness in the two cases.

In [1]:
import ross as rs
import numpy as np

In [2]:
# Define shaft material (steel)
steel = rs.Material(name="Steel", rho=7800, E=200e9, Poisson=0.27)

# Shaft length and number of elements
shaft_len = 1.6
n_elem = 16
elem_len = shaft_len / n_elem
speed = 3000 # Shaft spinning speed [rev/min] [rpm]

# Shaft geometries
# (a) Solid shaft: outer diameter = 60 mm
od_solid = 0.06
id_solid = 0.0

# (b) Hollow shaft: outer diameter = 80 mm, inner diameter = 73 mm
od_hollow = 0.08
id_hollow = 0.073

# Disk geometry
node_positions = [5, 10]         # Nodes where disks are located
i_d = 0.06                       # Inner diameter [m]
o_d = 0.4                        # Outer diameter [m]
thickness = 0.08                 # Disk thickness [m]

In [3]:
# Function to create shaft elements
def create_shaft_elements(id_d, od_d, shear, inertia):
    return [
        rs.ShaftElement(
            L=elem_len,
            idl=id_d,
            odl=od_d,
            material=steel,
            shear_effects=shear,
            rotary_inertia=inertia,
            gyroscopic=True
        )
        for _ in range(n_elem)
    ]

# Shaft Elements for Timoshenko model
shaft_solid_tim = create_shaft_elements(id_solid, od_solid, shear=True, inertia=True)
shaft_hollow_tim = create_shaft_elements(id_hollow, od_hollow, shear=True, inertia=True)

# Shaft Elements for Euler-Bernoulli model
shaft_solid_eb = create_shaft_elements(id_solid, od_solid, shear=False, inertia=False)
shaft_hollow_eb = create_shaft_elements(id_hollow, od_hollow, shear=False, inertia=False)

# Create identical disks according to node_positions input using from_geometry
disks = [rs.DiskElement.from_geometry(n=n, material=steel, width=thickness, i_d=i_d, o_d=o_d) for n in node_positions]

# Simple bearings at the ends
bearings = [rs.BearingElement(n=0, kxx=1e6, cxx=0),
            rs.BearingElement(n=n_elem, kxx=1e6, cxx=0)]

# Function to build and analyze rotor
def analyze_rotor(shaft_elements, disks, bearings, tag):
    rotor = rs.Rotor(shaft_elements=shaft_elements, disk_elements=disks, bearing_elements=bearings)
    modal = rotor.run_modal(speed= 2*np.pi*speed/60)
    print(f"\n{tag} - First 5 natural frequencies [Hz]:")
    print(np.round(modal.wn[:5] / (2 * np.pi), 2))
    return modal

# Compute percentage difference between models
def relative_difference(m1, m2):
    print("\nRelative difference [%]:")
    print(np.round(100 * np.abs((m1.wn[:5] - m2.wn[:5]) / m2.wn[:5]), 2))
    return #100 * np.abs((m1.wn[:5] - m2.wn[:5]) / m2.wn[:5])

# Analyses
print("="*70)
print("COMPARATIVE ANALYSES - SOLID SHAFT (TIMONSHENKO | EULER-BERNOULLI)")
print("="*70)

modal_solid_tim = analyze_rotor(shaft_solid_tim, disks, bearings, "Solid - Timoshenko")
modal_solid_eb = analyze_rotor(shaft_solid_eb, disks, bearings, "Solid - Euler-Bernoulli")
relative_difference_solid = relative_difference(modal_solid_tim, modal_solid_eb)

print("\n")
print("="*70)
print("COMPARATIVE ANALYSES - HOLLOW SHAFT (TIMONSHENKO | EULER-BERNOULLI)")
print("="*70)
modal_hollow_tim = analyze_rotor(shaft_hollow_tim, disks, bearings, "Hollow - Timoshenko")
modal_hollow_eb = analyze_rotor(shaft_hollow_eb, disks, bearings, "Hollow - Euler-Bernoulli")
relative_difference_hollow = relative_difference(modal_hollow_tim, modal_hollow_eb)


COMPARATIVE ANALYSES - SOLID SHAFT (TIMONSHENKO | EULER-BERNOULLI)

Solid - Timoshenko - First 5 natural frequencies [Hz]:
[11.62 11.87 32.13 39.86 80.31]

Solid - Euler-Bernoulli - First 5 natural frequencies [Hz]:
[11.63 11.87 32.17 39.94 80.31]

Relative difference [%]:
[0.06 0.07 0.12 0.21 0.  ]


COMPARATIVE ANALYSES - HOLLOW SHAFT (TIMONSHENKO | EULER-BERNOULLI)

Hollow - Timoshenko - First 5 natural frequencies [Hz]:
[12.14 12.42 35.24 44.4  79.07]

Hollow - Euler-Bernoulli - First 5 natural frequencies [Hz]:
[12.18 12.47 35.52 45.05 79.07]

Relative difference [%]:
[0.36 0.38 0.79 1.45 0.  ]
